# Capstone Notebook

# Stakeholder and Business Problem

This project is intended to provide the Minnesota Department of Transportation (MNDOT) with information and recommendations about what factors lead to heavy traffic volume and if anything can be done along I-94 to mitigate these factors. The dataset was pulled from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume#).

In [1]:
# Import tools and libraries
import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1)

# try:
    # %load_ext autotime
# except:
    # !pip install ipython-autotime
    # %load_ext autotime
    
%load_ext autotime

time: 0 ns (started: 2023-04-11 15:59:17 -06:00)


In [2]:
df_daily = pd.read_csv('data/complete_daily_values.csv')
df_daily.set_index('date', inplace=True)
df_daily

,temp,rain_in,snow_in,pct_cloud_cover,traffic_volume,holiday,year,month_day,month,day,weekday
date,,,,,,,,,,,
2012-10-02,63.056000,0.0,0.0,29.133333,63289.0,None,2012,10-02,10,2,Tuesday
2012-10-03,55.874300,0.0,0.0,3.850000,66345.0,None,2012,10-03,10,3,Wednesday
2012-10-04,61.173500,0.0,0.0,16.708333,89939.0,None,2012,10-04,10,4,Thursday
2012-10-05,48.070727,0.0,0.0,75.000000,93336.0,None,2012,10-05,10,5,Friday
2012-10-06,40.272957,0.0,0.0,61.652174,74910.0,None,2012,10-06,10,6,Saturday
...,...,...,...,...,...,...,...,...,...,...,...
2018-09-26,51.522500,0.0,0.0,39.833333,88627.0,None,2018,09-26,9,26,Wednesday
2018-09-27,55.613120,0.0,0.0,61.200000,94434.0,None,2018,09-27,9,27,Thursday
2018-09-28,47.189000,0.0,0.0,26.250000,92518.0,None,2018,09-28,9,28,Friday


time: 47 ms (started: 2023-04-11 15:59:17 -06:00)


# Baseline Model: Shifted

To start, we'll make a baseline model with just the dates and traffic volume values.

In [3]:
df_baseline = df_daily['traffic_volume']
df_baseline

date
2012-10-02    63289.0
2012-10-03    66345.0
2012-10-04    89939.0
2012-10-05    93336.0
2012-10-06    74910.0
               ...   
2018-09-26    88627.0
2018-09-27    94434.0
2018-09-28    92518.0
2018-09-29    76242.0
2018-09-30    68785.0
Name: traffic_volume, Length: 2190, dtype: float64

time: 15 ms (started: 2023-04-11 15:59:17 -06:00)


In [4]:
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error

# Create lagged dataset
baseline_values = DataFrame(df_baseline.values)
baseline_dataframe = concat([baseline_values.shift(1), baseline_values], axis=1)
baseline_dataframe.columns = ['t-1', 't+1']
print(baseline_dataframe.head(5))

       t-1      t+1
0      NaN  63289.0
1  63289.0  66345.0
2  66345.0  89939.0
3  89939.0  93336.0
4  93336.0  74910.0
time: 0 ns (started: 2023-04-11 15:59:17 -06:00)


In [5]:
# walk-forward validation
baseline_predictions = []

# split into train and test sets
baseline_X = baseline_dataframe.values
baseline_train_size = int(len(baseline_X) * 0.8)
baseline_train = baseline_dataframe[:baseline_train_size]
baseline_test = baseline_dataframe[baseline_train_size:]

# Looping through all values in test column 't+1'
for x in baseline_test['t+1']:
    y_hat = x
    baseline_predictions.append(x)
len(baseline_predictions)

438

time: 0 ns (started: 2023-04-11 15:59:17 -06:00)


In [6]:
# Calculating error for test column 't-1' against predictions (test column 't+1')
baseline_mae_score = mean_absolute_error(baseline_test['t-1'], baseline_predictions)
baseline_mse_score = mean_squared_error(baseline_test['t-1'], baseline_predictions)

print('Baseline MAE: %.3f' % baseline_mae_score)
print('Baseline MSE: %.3f' % baseline_mse_score)
print('Baseline RMSE: %.3f' % np.sqrt(baseline_mse_score))

Baseline MAE: 25019.169
Baseline MSE: 1385776125.539
Baseline RMSE: 37226.014
time: 0 ns (started: 2023-04-11 15:59:17 -06:00)


Mean Absolute Error (MAE) is a measure of the average difference between the predicted and test values. For our baseline model, the MAE is 25,020, suggesting our predicted traffic volume values, on average, differ from their actual values by 25,020 vehicles.

Root Mean Squared Error (RMSE) is a measure of the square root of the average squared difference between the predicted and test values. For our baseline model, the RMSE is 37,226.

In [7]:
# plot predictions and expected results
# fig, ax = plt.subplots(figsize=(14,10))

# plt.plot(baseline_train.index, baseline_train['t-1'], label='X_train')
# plt.plot(baseline_test.index, baseline_test['t-1'], label='X_test')
# plt.plot(baseline_test.index, baseline_predictions, label='Baseline Forecast')
# plt.legend(loc='best')
# plt.title('Baseline Forecast')
# plt.show()

time: 0 ns (started: 2023-04-11 15:59:17 -06:00)


# First Simple Model: ARIMA

In [8]:
# Define train and test sets
arima_train_size = int(len(df_baseline) * 0.8)

arima_train = df_baseline[:arima_train_size]
arima_test = df_baseline[arima_train_size:]

time: 0 ns (started: 2023-04-11 15:59:17 -06:00)


In [9]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(arima_train, label='train')
ax.plot(arima_test, label='test')
ax.set_title('Train-Test Split');
plt.legend();

Error in callback <function flush_figures at 0x0000022156AB48B0> (for post_execute):


KeyboardInterrupt: 

time: 15.4 s (started: 2023-04-11 15:59:17 -06:00)


In [10]:
from statsmodels.tsa.stattools import adfuller
from numpy import log

ad_test = adfuller(df_baseline)[1]
print(f"The p-value associated with the Dickey-Fuller statistical test is {ad_test},")
if ad_test < 0.05:
    print(" therefore we can safely assume that the dataset is stationary.")
else:
    print(" therefore we cannot reject the null hypothesis that the dataset is \
not stationary.")

The p-value associated with the Dickey-Fuller statistical test is 1.4989446630712853e-06,
 therefore we can safely assume that the dataset is stationary.
time: 109 ms (started: 2023-04-11 15:59:32 -06:00)


After checking for stationarity, we can proceed with building a basic ARIMA model. The Auto-ARIMA function was used in a Google Colab notebook to determine the best parameters for our dataset.

![auto_arima_results](images/auto_arima_results.png)

In [11]:
from statsmodels.tsa.arima.model import ARIMA

# Build Model
arima_model = ARIMA(arima_train, order=(1, 0, 3)).fit()

# Predictions from model
arima_pred = arima_model.predict(start = arima_test.index[0], end = arima_test.index[-1])

C:\Users\pschu\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\pschu\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\pschu\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\pschu\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\pschu\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tsa\

time: 984 ms (started: 2023-04-11 15:59:33 -06:00)


In [12]:
# MAE, MSE, and RMSE scores
arima_mae_score = mean_absolute_error(arima_test, arima_pred)
arima_mse_score = mean_squared_error(arima_test, arima_pred)

print('ARIMA MAE: %.3f' % arima_mae_score)
print('ARIMA MSE: %.3f' % arima_mse_score)
print('ARIMA RMSE: %.3f' % np.sqrt(arima_mse_score))

ARIMA MAE: 20469.183
ARIMA MSE: 1016465010.612
ARIMA RMSE: 31882.048
time: 16 ms (started: 2023-04-11 15:59:34 -06:00)


The MAE has gone from 25,020 in our baseline model to 20,470 and our RMSE has gone from 37,226 to 31,882 in our first simple model. An improvement!

In [13]:
# Predicting data
y_pred = arima_model.get_forecast(len(arima_test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df['Predictions'] = arima_model.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = arima_test.index
y_pred_out = y_pred_df['Predictions']

# Forecasting future data
arima_output = arima_model.predict(start='2018-10-01',end='2019-09-30')

time: 62 ms (started: 2023-04-11 15:59:34 -06:00)


In [14]:
fig, ax = plt.subplots(figsize=(14, 10))
# ax.plot(train, label='train')
ax.plot(arima_test, label='test')
ax.plot(y_pred_out, label='forecast')
ax.plot(arima_output, label='predictions')

plt.xlabel('Date', size=22)
plt.xticks(size=20, rotation=45)
plt.ylabel('Traffic Volume', size=22)
plt.yticks(size=20)
plt.title('Test Forecast and Model Predictions', size=24)

plt.legend();

TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'UnitData'

TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'UnitData'

<Figure size 1008x720 with 1 Axes>

time: 204 ms (started: 2023-04-11 15:59:34 -06:00)


## OLS Model: As-Is and Standardized

In [ ]:
# Converting holiday column to binary integers
# df_daily[df_daily['holiday'] == 'None'] = 0
# df_daily[df_daily['holiday'] != 0] = 1

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Create model with all numeric variables
X = df_daily[['temp', 'rain_in', 'snow_in', 'pct_cloud_cover']]
y = df_daily['traffic_volume']

ols_model = sm.OLS(y, sm.add_constant(X)).fit()
ols_model.summary()

In [ ]:
ols_model.params

In [ ]:
np.sqrt(ols_model.mse_resid)

In [ ]:
# Create standardized dataframe to get all variables on the same scale
X_standardized = df_daily[['temp', 'rain_in', 'snow_in', 'pct_cloud_cover']]

for col in X_standardized:
    X_standardized[col] = (X_standardized[col] - X_standardized[col].mean()) \
                            / X_standardized[col].std()
    
X_standardized.describe()

In [ ]:
# Create standardized model
X = X_standardized
y = df_daily['traffic_volume']

ols_model_std = sm.OLS(endog=y, exog=sm.add_constant(X)).fit()

ols_model_std.summary()

In [ ]:
ols_model_std.params

In [ ]:
np.sqrt(ols_model_std.mse_resid)

# Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_daily.drop(['year', 'month', 'day'], axis=1).corr(),annot=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.regplot(x=df_daily['pct_cloud_cover'], y=df_daily['traffic_volume'], line_kws={"color":"r","alpha":0.7,"lw":5})
plt.show()

In [ ]:
graph = df_daily.groupby(['month']).agg({'traffic_volume':'mean'}).sort_values(by='traffic_volume', ascending=False)

In [ ]:
graph

In [ ]:
fig, ax = plt.subplots(figsize = (12 , 10))

fig = sns.barplot(data = graph,
                  x = graph.index,
                  y = 'traffic_volume')

fig.axhline(df_daily['traffic_volume'].mean())

plt.show();

In [ ]:
df_daily

In [ ]:
cloud_mean = df_daily.groupby('pct_cloud_cover').mean()
cloud_mean.reset_index(drop=False, inplace=True)

In [ ]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80',  '81-90',  '91-100']
cloud_mean['bin'] = pd.cut(x = cloud_mean['pct_cloud_cover'], bins = bins, labels = labels, include_lowest = True)

In [ ]:
cloud_mean_bin = cloud_mean.groupby('bin').mean()
cloud_mean_bin.reset_index(drop=False, inplace=True)
cloud_mean_bin

In [ ]:
fig, ax = plt.subplots(figsize = (12 , 10))

fig = sns.barplot(data = cloud_mean_bin,
                  x = 'bin',
                  y = 'traffic_volume')

fig.axhline(df_daily['traffic_volume'].mean())

plt.xlabel('Percent Cloud Cover', size=22)
plt.xticks(size=20, rotation=45)
plt.ylabel('Mean Traffic Volume', size=22)
plt.yticks(size=20)
plt.title('Mean Traffic Volume by Percent Cloud Cover', size=24)
plt.show(fig);

In [ ]:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    
# Call the function and run the test
adf_test(df_baseline)

In [ ]:
from statsmodels.tsa.stattools import kpss

def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c', nlags="auto")
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','#Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)
    
kpss_test(df_baseline)